# Extracting Youtube comments from YouTube API

In [ ]:
pip install --upgrade google-api-python-client

     |████████████████████████████████| 8.0 MB 14.7 MB/s 
  Attempting uninstall: google-auth-httplib2
    Found existing installation: google-auth-httplib2 0.0.4
    Uninstalling google-auth-httplib2-0.0.4:
      Successfully uninstalled google-auth-httplib2-0.0.4
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.10
    Uninstalling google-api-python-client-1.12.10:
      Successfully uninstalled google-api-python-client-1.12.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.296 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 2.36.0 which is incompatible.


In [ ]:
from apiclient.discovery import build

In [ ]:
key = 'xxxx'
videoId = '0u_zNDQIeHk'

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 300)

In [ ]:
def build_service():
    YOUTUBE_API_SERVICE_NAME = "youtube"
    YOUTUBE_API_VERSION = "v3"
    return build(YOUTUBE_API_SERVICE_NAME,
                 YOUTUBE_API_VERSION,
                 developerKey=key)

In [ ]:
# configure function parameters for required variables to pass to service
def get_comments( part='snippet', videoId=videoId, maxResults=100, textFormat='plainText', order='time'):
    
    # build our service from path/to/apikey
    service = build_service()
    
    # make an API call using our service
    response = service.commentThreads().list(
        part='snippet', 
        videoId=videoId,                      
        maxResults=100, 
        textFormat='plainText',
        order='time'
    ).execute()

    # create empty lists to store desired information
    comments, commentsId, authorurls, authornames, repliesCount, likesCount, viewerRating, dates, totalReplyCounts = [], [], [], [], [], [], [], [], []

    while response: # this loop will continue to run until you max out your quota

        for item in response['items']:
            # index item for desired data features
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comment_id = item['snippet']['topLevelComment']['id']
            reply_count = item['snippet']['totalReplyCount']
            like_count = item['snippet']['topLevelComment']['snippet']['likeCount']
            authorurl = item['snippet']['topLevelComment']['snippet']['authorChannelUrl']
            authorname = item['snippet']['topLevelComment']['snippet']['authorDisplayName']
            date = item['snippet']['topLevelComment']['snippet']['publishedAt']
            totalReplyCount = item['snippet']['totalReplyCount']

            # append to lists
            comments.append(comment)
            commentsId.append(comment_id)
            repliesCount.append(reply_count)
            likesCount.append(like_count)
            authorurls.append(authorurl)
            authornames.append(authorname)
            dates.append(date)
            totalReplyCounts.append(totalReplyCount)

        try:
            if 'nextPageToken' in response:
                response = service.commentThreads().list(
                    part=part,
                    maxResults=maxResults,
                    textFormat=textFormat,
                    order=order,
                    pageToken=response['nextPageToken']
                ).execute()
            else:
                break
        except: break

    #9 return our data of interest
    return {
        'comment': comments,
        'comment_id': commentsId,
        'author_url': authorurls,
        'author_name': authornames,
        'reply_count' : repliesCount,
        'like_count' : likesCount,
        'date': dates,
        'total_reply_counts': totalReplyCounts
    }

In [ ]:
GOT_comments = get_comments()
df = pd.DataFrame(GOT_comments)
print(df.shape)

(100, 8)


In [ ]:
df.head(5)

,comment,comment_id,author_url,author_name,reply_count,like_count,date,total_reply_counts
0,Ur the most optimistic person in the universe and if you're piss Ed it means something,Ugylute2Ab6mA0G2cSF4AaABAg,http://www.youtube.com/channel/UCr8U7Y4Y4sBbrLCWrW7cbaQ,HJRC_0823,1,0,2022-02-08T07:52:58Z,1
1,"It wasn't just the direction that plot moved, did you also note how the dialog stopped including poetic and thoughtful lines and how Varys and Tyrion's wit and intelligence dropped from magnficant to average dunderhead",UgyU8Y348xtxK5_LlGZ4AaABAg,http://www.youtube.com/channel/UCDbXCvRxqXdrPcaV-mwxpnQ,Furry Lovebear,0,0,2022-01-25T22:23:47Z,0
2,Lazy writing towards the end.,UgxNEHssvL6RlJ9KQex4AaABAg,http://www.youtube.com/channel/UCzL_W1HNv69xdPSI2_LgLXQ,Charles Comeaux jr,0,0,2022-01-18T23:32:29Z,0
3,Cool,Ugy638KYY2rLNdovh2V4AaABAg,http://www.youtube.com/channel/UCKxf8_MDBHKpz5a-K10rAhw,Beavis Johnson,0,0,2022-01-12T05:51:08Z,0
4,The night king vs people was the main plot point they just never wanted to address it. Was the first scene in the show and the last in every season,UgzJeAc28XQ3jCENR9B4AaABAg,http://www.youtube.com/channel/UCFJqObk82C8YaemPFoBucUw,Luke Coupland,0,0,2022-01-08T02:09:21Z,0


In [ ]:
df.author_name.value_counts(), df.author_url.value_counts()

(Niraj Kathayat     3
 Gammermonk         2
 Tyler Anselmo      2
 Michael            2
 Furry Lovebear     1
                   ..
 Daniel Krawczyk    1
 Rajaz Khan         1
 jp turchi          1
 Wellsigood         1
 Chris M            1
 Name: author_name, Length: 95, dtype: int64,
 http://www.youtube.com/channel/UCNHEQ6s95c5WWE7UDJJNwWQ    3
 http://www.youtube.com/channel/UCuUqw4CFUU2rCfT1AfxBBQQ    2
 http://www.youtube.com/channel/UCjl_1uDDSJaYgouhf3JJHbg    2
 http://www.youtube.com/channel/UCDbXCvRxqXdrPcaV-mwxpnQ    1
 http://www.youtube.com/channel/UCh9_WIsk4v-_VU_BrqYJV4Q    1
                                                           ..
 http://www.youtube.com/channel/UCG9JqMvZCY6dgRYkUJZMQyw    1
 http://www.youtube.com/channel/UCKyUPqSSoserAxzKCRM7sAA    1
 http://www.youtube.com/channel/UCMCnoQKb9pYEpCfI1qO3GzA    1
 http://www.youtube.com/channel/UCBYjgxDJ6XyPP5GYQSb1FCg    1
 http://www.youtube.com/channel/UCwvqq1w9ffDhJkidZnrC5Hw    1
 Name: author_url, Length: 96,

Looks like we do not need the author_url column to uniquely identify the authors as the name column seem to do that on its own. Therefore, we can consider dropping author_url column.

In [ ]:
df.drop('author_url', axis=1, inplace=True)
df.head(5)

,comment,comment_id,author_name,reply_count,like_count,date,total_reply_counts
0,Ur the most optimistic person in the universe and if you're piss Ed it means something,Ugylute2Ab6mA0G2cSF4AaABAg,HJRC_0823,1,0,2022-02-08T07:52:58Z,1
1,"It wasn't just the direction that plot moved, did you also note how the dialog stopped including poetic and thoughtful lines and how Varys and Tyrion's wit and intelligence dropped from magnficant to average dunderhead",UgyU8Y348xtxK5_LlGZ4AaABAg,Furry Lovebear,0,0,2022-01-25T22:23:47Z,0
2,Lazy writing towards the end.,UgxNEHssvL6RlJ9KQex4AaABAg,Charles Comeaux jr,0,0,2022-01-18T23:32:29Z,0
3,Cool,Ugy638KYY2rLNdovh2V4AaABAg,Beavis Johnson,0,0,2022-01-12T05:51:08Z,0
4,The night king vs people was the main plot point they just never wanted to address it. Was the first scene in the show and the last in every season,UgzJeAc28XQ3jCENR9B4AaABAg,Luke Coupland,0,0,2022-01-08T02:09:21Z,0


In [ ]:
df.shape

(100, 7)

In [ ]:
# saving the csv file
df.to_csv('./got_comments.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')